In [20]:
! pip install python-docx
! pip install spacy_annotator

In [21]:
from docx import Document
from os import walk
from tqdm.notebook import tqdm
import spacy_annotator
import pandas as pd
import spacy

In [24]:
def get_text(fn, folder_path):
  fref = open(folder_path + fn, "rb")
  doc = Document(folder_path + fn)
  text = []
  for para in doc.paragraphs:
    text.append(para.text)
    #text += para.text + "\n"

  return '\n'.join(text)

def load_hudoc(folder_path):
  counter = 0
  break_counter = 0
  texts = []
  fns = next(walk(folder_path), (None, None, []))[2]  # [] if no file
  for fn in tqdm(fns):
    try:
      texts.append(get_text(fn, folder_path))
      break_counter += 1
    except:
      counter += 1

    if break_counter > 100:
      break

  print("{} files not extracted.".format(counter))

  return texts

def clean_file(courts):
  return [court.replace('\xa0', ' ') for court in courts]

def annotate_by_sentence(courts, nlp):
  annotated_data = []

  # For every court
  for court in tqdm(courts):
    annotated_court = []

    # Get all individual sentences (using spaCy tokenizer)
    sentences = [str(x) for x in nlp(court).sents]

    # For every sentence
    for sentence in sentences:
      annotated_sentence = []
      for token in nlp(sentence):
        annotated_sentence.append((token, token.ent_iob_, token.ent_type_))

      # Add sentence to court list
      annotated_court.append(annotated_sentence)

    # Add court to all data
    annotated_data.append(annotated_court)

  return annotated_data

In [25]:
import sys

courts = load_hudoc('../Files/')
courts = clean_file(courts)

1282 files not extracted.


  0%|          | 0/20166 [00:00<?, ?it/s]

59 files not extracted.


In [45]:
print(courts[0])




FOURTH SECTION





CASE OF POHOSKA v. 

(Application no. 33530/06)








JUDGMENT




10 January 2012

FINAL

10/04/2012

This judgment has become final under Article 44 § 2 of the Convention. It may be subject to editorial revision.
In the case of Pohoska v. ,
The European Court of Human Rights (Fourth Section), sitting as a Chamber composed of:
	David Thór Björgvinsson, President,
	Lech Garlicki,
	George Nicolaou,
	Ledi Bianku,
	Zdravka Kalaydjieva,
	Nebojša Vučinić,
	Vincent A. De Gaetano, judges,
and Fatoş Aracı, Deputy Section Registrar,
Having deliberated in private on 6 December 2011,
Delivers the following judgment, which was adopted on that date:
PROCEDURE
1.  The case originated in an application (no. 33530/06) against the  lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Polish national, Ms Danuta Pohoska (“the applicant”), on 25 July 2006.
2.  The applicant, who had been grante

In [26]:
# Load model
nlp = spacy.load("en_core_web_sm")

In [27]:
annotated_data = annotate_by_sentence(courts, nlp)

  0%|          | 0/101 [00:00<?, ?it/s]

In [30]:
print(courts[0])
print(annotated_data[0])




FOURTH SECTION





CASE OF POHOSKA v. 

(Application no. 33530/06)








JUDGMENT




10 January 2012

FINAL

10/04/2012

This judgment has become final under Article 44 § 2 of the Convention. It may be subject to editorial revision.
In the case of Pohoska v. ,
The European Court of Human Rights (Fourth Section), sitting as a Chamber composed of:
	David Thór Björgvinsson, President,
	Lech Garlicki,
	George Nicolaou,
	Ledi Bianku,
	Zdravka Kalaydjieva,
	Nebojša Vučinić,
	Vincent A. De Gaetano, judges,
and Fatoş Aracı, Deputy Section Registrar,
Having deliberated in private on 6 December 2011,
Delivers the following judgment, which was adopted on that date:
PROCEDURE
1.  The case originated in an application (no. 33530/06) against the  lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Polish national, Ms Danuta Pohoska (“the applicant”), on 25 July 2006.
2.  The applicant, who had been grante

In [23]:
# Change format of dataset starting here.
import pandas as pd
import numpy as np

training_sentence = []
sentence_count = 1
training_text = []
training_labels = []
pos = []
for section in annotated_data:
  for entry in section:

    training_sentence.append("Sentence: " + str(sentence_count))
    flag = False

    for data in entry:
      # text data to clean
      text_data = str(data[0])
      text_data = text_data.replace("\n", "")
      text_data = text_data.replace("\t", "")

      # Find escape characters to clean
      if "\\" in text_data:
        print(text_data)

      # Update sentence tracking
      if flag:
        training_sentence.append(np.nan)
      else:
        flag = True

      training_text.append(text_data)
      training_labels.append(str(data[1]) + ' ' + str(data[2]))
      pos.append("NNS")
      #training_data.append(training_entry)

    #break
    #if sentence_count >= 2:
    #  break
    sentence_count += 1

  #break

print(len(training_sentence))
print(len(training_text))
print(len(training_labels))

1536\00
2415\00
128\382
17761190
17761190
17761190


In [25]:
#aggregated = [training_sentence, training_text, training_labels]
df = pd.DataFrame({
  "Sentence #": training_sentence,
  "Word": training_text,
  "POS": pos,
  "Tag": training_labels
                   })
df.to_csv("kaggle_at_home.csv")

In [26]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,,NNS,O
1,NaN,FOURTH,NNS,O
2,NaN,SECTION,NNS,O
3,NaN,,NNS,O
4,NaN,CASE,NNS,O


In [ ]:
import pandas as pd

training_data = []
i = 0
for section in annotated_data:
  for entry in section:
    for data in entry:
      # text data to clean
      text_data = str(data[0])
      text_data = text_data.replace("\n", "")
      text_data = text_data.replace("\t", "")

      # Find escape characters to clean
      if "\\" in text_data:
        print(text_data)

      training_entry = text_data + ' ' + str(data[1]) + ' ' + str(data[2])
      training_data.append(training_entry)

In [10]:
# Save as DF
train_df = pd.DataFrame(training_data, columns=[["Entities"]])
train_df.head()

train_df.to_csv("train_df10000.csv")

In [172]:
## Do the processing for Jan's dataset

data = pd.read_csv("../file002.csv")
print(len(data))

# Load model
nlp = spacy.load("en_core_web_sm")


135


In [173]:
def annotate_by_sentence2(courts, nlp):
  annotated_data = []

  # For every court
  for court in tqdm(courts):
    annotated_court = []

    #print(court)

    # Remake sentence


    # Get all individual sentences (using spaCy tokenizer)
    sentences = [str(x) for x in nlp(court).sents]

    # For every sentence
    for sentence in sentences:
      annotated_sentence = []
      for token in nlp(sentence):
        annotated_sentence.append((token, token.ent_iob_, token.ent_type_))

      # Add sentence to court list
      annotated_court.append(annotated_sentence)

    # Add court to all data
    annotated_data.append(annotated_court)

  return annotated_data


In [174]:
# Get the proper text
word_data = data['Word']
# Transform into normal text
print(word_data[0])
new_word = []

for sent in word_data:
  new_word.append(sent.replace("['", '').replace("']", "").replace("', '", " "))

print(new_word)

['THIRD', 'SECTION']
['THIRD SECTION', 'CASE OF BUCEA AND BUCIA v. ROMANIA', '( Application no . 32185/04 )', 'JUDGMENT', 'This version was rectified on 16 December 2014 under Rule 81 of the Rules of Court', 'STRASBOURG', '1 July 2014', 'FINAL', '01/10/2014', 'This judgment has become final under Article 44 § 2 of the Convention . It may be subject to editorial revision .', 'In the case of Bucea and Bucia v. Romania ,', 'The European Court of Human Rights ( Third Section ) , sitting as a Chamber compomsed of :', 'Josep Casadevall , President , Alvina Gyulumyan , J n ikuta , Luis L pez Guerra , Johannes Silvis , Valeriu Gri co , Iulia Antoanella Motoc , judges , and Santiago Quesada , Section Registrar ,', 'Having deliberated in private on 10 June 2014 ,', 'Delivers the following judgment , which was adopted on that date :', 'PROCEDURE', 'The case originated in an application ( no . 32185/04 ) against Romania lodged with the Court under Article 34 of the Convention for the Protection of

In [175]:
annotated_data = annotate_by_sentence2(new_word, nlp)

  0%|          | 0/135 [00:00<?, ?it/s]

In [176]:
print(annotated_data)


[[[(THIRD, 'B', 'ORG'), (SECTION, 'I', 'ORG')]], [[(CASE, 'O', ''), (OF, 'O', ''), (BUCEA, 'O', ''), (AND, 'O', ''), (BUCIA, 'B', 'ORG'), (v., 'O', ''), (ROMANIA, 'O', '')]], [[((, 'O', ''), (Application, 'O', ''), (no, 'O', ''), (., 'O', ''), (32185/04, 'B', 'CARDINAL'), (), 'O', '')]], [[(JUDGMENT, 'O', '')]], [[(This, 'O', ''), (version, 'O', ''), (was, 'O', ''), (rectified, 'O', ''), (on, 'O', ''), (16, 'B', 'DATE'), (December, 'I', 'DATE'), (2014, 'I', 'DATE'), (under, 'O', ''), (Rule, 'B', 'LAW'), (81, 'I', 'LAW'), (of, 'O', ''), (the, 'O', ''), (Rules, 'O', ''), (of, 'O', ''), (Court, 'O', '')]], [[(STRASBOURG, 'B', 'ORG')]], [[(1, 'O', ''), (July, 'B', 'DATE'), (2014, 'I', 'DATE')]], [[(FINAL, 'O', '')]], [[(01/10/2014, 'O', '')]], [[(This, 'O', ''), (judgment, 'O', ''), (has, 'O', ''), (become, 'O', ''), (final, 'O', ''), (under, 'O', ''), (Article, 'B', 'LAW'), (44, 'I', 'LAW'), (§, 'O', ''), (2, 'O', ''), (of, 'O', ''), (the, 'O', ''), (Convention, 'O', ''), (., 'O', '')], [

In [177]:
# Transform it all into the right format:
# Change format of dataset starting here.
import pandas as pd
import numpy as np

training_sentence = []
sentence_count = 1
training_text = []
training_labels = []
pos = []
for section in annotated_data:
  for entry in section:

    training_sentence.append("Sentence: " + str(sentence_count))
    flag = False

    for datae in entry:
      # text data to clean
      text_data = str(datae[0])
      text_data = text_data.replace("\n", "")
      text_data = text_data.replace("\t", "")

      # Find escape characters to clean
      if "\\" in text_data:
        print(text_data)

      # Update sentence tracking
      if flag:
        training_sentence.append(np.nan)
      else:
        flag = True

      training_text.append(text_data)
      training_labels.append(str(datae[1]) + ' ' + str(datae[2]))
      pos.append("NNS")
      #training_data.append(training_entry)

    #break
    #if sentence_count >= 2:
    #  break
    sentence_count += 1

  #break

print(len(training_sentence))
print(len(training_text))
print(len(training_labels))

5447
5447
5447


In [178]:
#aggregated = [training_sentence, training_text, training_labels]
df = pd.DataFrame({
  "Sentence #": training_sentence,
  "Word": training_text,
  "POS": pos,
  "Tag": training_labels
                   })

#print(df.head())

# Reloop to turn back to the other format:

spacy_data = []
paragraph = []
p_word = []
text = ""
p_tag = []
p_text = []
p_pos = []
for index, row in df.iterrows():
  #print(str(row['Sentence #']))
  if str(row['Sentence #']) != "nan":
    if len(p_word) > 0:
      spacy_data.append([text, p_word, p_pos, p_tag])
      text = ""
      p_word = []
      p_tag = []
      p_pos = []

  text += row['Word'] + " "
  p_word.append(row['Word'])
  p_tag.append(row['Tag'])
  p_pos.append('NNS')

  #break

In [179]:
print(spacy_data[0])
spacy_df = pd.DataFrame(spacy_data, columns=["Text", "Word", "POS", "Tag"])
print(spacy_df.head())

['THIRD SECTION ', ['THIRD', 'SECTION'], ['NNS', 'NNS'], ['B ORG', 'I ORG']]
                                                Text  \
0                                     THIRD SECTION    
1                CASE OF BUCEA AND BUCIA v. ROMANIA    
2                     ( Application no . 32185/04 )    
3                                          JUDGMENT    
4  This version was rectified on 16 December 2014...   

                                                Word  \
0                                   [THIRD, SECTION]   
1         [CASE, OF, BUCEA, AND, BUCIA, v., ROMANIA]   
2               [(, Application, no, ., 32185/04, )]   
3                                         [JUDGMENT]   
4  [This, version, was, rectified, on, 16, Decemb...   

                                                 POS  \
0                                         [NNS, NNS]   
1                [NNS, NNS, NNS, NNS, NNS, NNS, NNS]   
2                     [NNS, NNS, NNS, NNS, NNS, NNS]   
3                        

In [180]:
print(data.head())

   Unnamed: 0                                               Text  \
0           0                                      THIRD SECTION   
1           1                 CASE OF BUCEA AND BUCIA v. ROMANIA   
2           2                         (Application no. 32185/04)   
3           3                                           JUDGMENT   
4           4  This version was rectified on 16 December 2014...   

                                                Word  POS  \
0                               ['THIRD', 'SECTION']  NaN   
1  ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', ...  NaN   
2   ['(', 'Application', 'no', '.', '32185/04', ')']  NaN   
3                                       ['JUDGMENT']  NaN   
4  ['This', 'version', 'was', 'rectified', 'on', ...  NaN   

                                                 Tag  Word_idx  \
0                             ['B-COURT', 'L-COURT']       NaN   
1  ['O', 'O', 'O', 'O', 'B-CASE', 'I-CASE', 'L-CA...       NaN   
2  ['O', 'O', 'B-APPLICATI

In [181]:
# Now do the regex
print(len(spacy_data))
print(len(data))


231
135


In [233]:
old_index = 0
data_redone = []

In [234]:
# Puts the same elements

index = old_index
while index < len(data):
  if str(spacy_df.iloc[index]['Word']) != str(data.iloc[index]['Word']):
    print(str(index) + " " + str(spacy_df.iloc[index]['Word']) + ", " + str(data.iloc[index]['Word']))

    s_index = 0
    all_sentences = []
    sub_sentence = []
    row_array = data.iloc[index]['Word'].replace("['", "").replace("']", "").split("', '")
    while s_index < len(row_array):
      sub_sentence.append(row_array[s_index])
      if row_array[s_index] == ".":
        print(s_index)
        all_sentences.append(sub_sentence)
        sub_sentence = []
      s_index += 1

    # reappend to normal array
    for arr in all_sentences:
      data_redone.append(arr)

    old_index = index + 1
    #break

  else:
    data_redone.append(spacy_df.iloc[index]['Word'])

  index += 1


9 ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.'], ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.', 'It', 'may', 'be', 'subject', 'to', 'editorial', 'revision', '.']
13
21
10 ['It', 'may', 'be', 'subject', 'to', 'editorial', 'revision', '.'], ['In', 'the', 'case', 'of', 'Bucea', 'and', 'Bucia', 'v.', 'Romania', ',']
11 ['In', 'the', 'case', 'of', 'Bucea', 'and', 'Bucia', 'v.', 'Romania', ','], ['The', 'European', 'Court', 'of', 'Human', 'Rights', '(', 'Third', 'Section', ')', ',', 'sitting', 'as', 'a', 'Chamber', 'compomsed', 'of', ':']
12 ['The', 'European', 'Court', 'of', 'Human', 'Rights', '(', 'Third', 'Section', ')', ',', 'sitting', 'as', 'a', 'Chamber', 'compomsed', 'of', ':'], ['Josep', 'Casadevall', ',', 'President', ',', 'Alvina', 'Gyulumyan', ',', 'J', 'n', 'ikuta', ',', 'Luis', 'L', 'pez', 'Guerra', ',', 'Johannes', 'Silvis', ',', 'Valeriu', 'G

In [235]:
print(len(data_redone))
print(old_index)
for i in range(len(data_redone)):
  print(str(i) + " " + str(data_redone[i]) + " " + str(spacy_df.iloc[i]['Word']))


186
135
0 ['THIRD', 'SECTION'] ['THIRD', 'SECTION']
1 ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', 'ROMANIA'] ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', 'ROMANIA']
2 ['(', 'Application', 'no', '.', '32185/04', ')'] ['(', 'Application', 'no', '.', '32185/04', ')']
3 ['JUDGMENT'] ['JUDGMENT']
4 ['This', 'version', 'was', 'rectified', 'on', '16', 'December', '2014', 'under', 'Rule', '81', 'of', 'the', 'Rules', 'of', 'Court'] ['This', 'version', 'was', 'rectified', 'on', '16', 'December', '2014', 'under', 'Rule', '81', 'of', 'the', 'Rules', 'of', 'Court']
5 ['STRASBOURG'] ['STRASBOURG']
6 ['1', 'July', '2014'] ['1', 'July', '2014']
7 ['FINAL'] ['FINAL']
8 ['01/10/2014'] ['01/10/2014']
9 ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.'] ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.']
10 ['It', 'may', 'be', 'subject', 'to', 'editorial', 'revision', '.'] ['It

In [167]:

print(data.iloc[10]['Word'])

s_index = 0
all_sentences = []
sub_sentence = []
row_array = data.iloc[9]['Word'].replace("['", "").replace("']", "").split("', '")
while s_index < len(row_array):
  sub_sentence.append(row_array[s_index])
  if row_array[s_index] == ".":
    print(s_index)
    all_sentences.append(sub_sentence)
    sub_sentence = []
  s_index += 1


['In', 'the', 'case', 'of', 'Bucea', 'and', 'Bucia', 'v.', 'Romania', ',']
13
21


In [165]:
for sent in range(len(spacy_df)):
  if str(spacy_df.iloc[sent]['Word']) != str(data_redone[sent]):
    print(str(sent) + " " + str(spacy_df.iloc[sent]['Word']) + " = " + str(data_redone[sent]))




11 ['In', 'the', 'case', 'of', 'Bucea', 'and', 'Bucia', 'v.', 'Romania', ','] = ['The', 'case', 'originated', 'in', 'an', 'application', '(', 'no', '.']
12 ['The', 'European', 'Court', 'of', 'Human', 'Rights', '(', 'Third', 'Section', ')', ',', 'sitting', 'as', 'a', 'Chamber', 'compomsed', 'of', ':'] = ['32185/04', ')', 'against', 'Romania', 'lodged', 'with', 'the', 'Court', 'under', 'Article', '34', 'of', 'the', 'Convention', 'for', 'the', 'Protection', 'of', 'Human', 'Rights', 'and', 'Fundamental', 'Freedoms', '(', 'the', 'Convention', ')', 'by', 'two', 'Romanian', 'nationals', ',', 'Mr', 'Ioan', 'Bucea', 'and', 'Mr', 'Alexandru', 'Bucia', '(', 'the', 'applicants', ')', ',', 'on', '15', 'July', '2004', '.']
13 ['Josep', 'Casadevall', ',', 'President', ',', 'Alvina', 'Gyulumyan', ',', 'J', 'n', 'ikuta', ',', 'Luis', 'L', 'pez', 'Guerra', ',', 'Johannes', 'Silvis', ',', 'Valeriu', 'Gri', 'co', ',', 'Iulia', 'Antoanella', 'Motoc', ',', 'judges', ',', 'and', 'Santiago', 'Quesada', ',', '

IndexError: list index out of range

In [230]:
for i in range(len(data_redone)):
  print(str(i) + " " + str(data_redone[i]) + " = " + str(spacy_df.iloc[i]['Word']))

0 ['THIRD', 'SECTION'] = ['THIRD', 'SECTION']
1 ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', 'ROMANIA'] = ['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', 'ROMANIA']
2 ['(', 'Application', 'no', '.', '32185/04', ')'] = ['(', 'Application', 'no', '.', '32185/04', ')']
3 ['JUDGMENT'] = ['JUDGMENT']
4 ['This', 'version', 'was', 'rectified', 'on', '16', 'December', '2014', 'under', 'Rule', '81', 'of', 'the', 'Rules', 'of', 'Court'] = ['This', 'version', 'was', 'rectified', 'on', '16', 'December', '2014', 'under', 'Rule', '81', 'of', 'the', 'Rules', 'of', 'Court']
5 ['STRASBOURG'] = ['STRASBOURG']
6 ['1', 'July', '2014'] = ['1', 'July', '2014']
7 ['FINAL'] = ['FINAL']
8 ['01/10/2014'] = ['01/10/2014']
9 ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.'] = ['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.']
10 ['It', 'may', 'be', 'subject', 'to', 'editorial', 'revision

In [232]:
print(str(data.iloc[12]['Word']) + " " + str(data.iloc[12]['Word']))

['Josep', 'Casadevall', ',', 'President', ',', 'Alvina', 'Gyulumyan', ',', 'J', 'n', 'ikuta', ',', 'Luis', 'L', 'pez', 'Guerra', ',', 'Johannes', 'Silvis', ',', 'Valeriu', 'Gri', 'co', ',', 'Iulia', 'Antoanella', 'Motoc', ',', 'judges', ',', 'and', 'Santiago', 'Quesada', ',', 'Section', 'Registrar', ','] ['Josep', 'Casadevall', ',', 'President', ',', 'Alvina', 'Gyulumyan', ',', 'J', 'n', 'ikuta', ',', 'Luis', 'L', 'pez', 'Guerra', ',', 'Johannes', 'Silvis', ',', 'Valeriu', 'Gri', 'co', ',', 'Iulia', 'Antoanella', 'Motoc', ',', 'judges', ',', 'and', 'Santiago', 'Quesada', ',', 'Section', 'Registrar', ',']
